In [7]:
import numpy as np
from numba import jit, prange
from numba import config
from numba import jit, cuda
#config.THREADING_LAYER = 'omp'

class ConvolutionLayer:
    def __init__(self, kernel_num, kernel_size):
        """
        Constructor takes as input the number of kernels and their size. I assume only squared filters of size kernel_size x kernel_size
        """
        self.kernel_num = kernel_num
        self.kernel_size = kernel_size
        # Generate random filters of shape (kernel_num, kernel_size, kernel_size). Divide by kernel_size^2 for weight normalization
        self.kernels = np.random.randn(kernel_num, kernel_size, kernel_size) / (kernel_size**2)

    def patches_generator(self, image):
        """
        Divide the input image in patches to be used during convolution.
        Yields the tuples containing the patches and their coordinates.
        """
        # Extract image height and width
        image_h, image_w = image.shape
        self.image = image
        # The number of patches, given a fxf filter is h-f+1 for height and w-f+1 for width
        patches = np.empty((image_h-self.kernel_size+1, image_w-self.kernel_size+1, self.kernel_size, self.kernel_size))
        for h in range(image_h-self.kernel_size+1):
            for w in range(image_w-self.kernel_size+1):
                patches[h, w] = image[h:(h+self.kernel_size), w:(w+self.kernel_size)]
        return patches

    def forward_prop(self, image):
        """
        Perform forward propagation for the convolutional layer.
        """
        # Extract image height and width
        image_h, image_w = image.shape
        # Initialize the convolution output volume of the correct size
        convolution_output = np.zeros((image_h-self.kernel_size+1, image_w-self.kernel_size+1, self.kernel_num))
        # Unpack the generator
        patches = self.patches_generator(image)
        block_size = (16, 16)
        grid_size = (math.ceil(convolution_output.shape[1] / block_size[0]),
                     math.ceil(convolution_output.shape[0] / block_size[1]))
        cnn_forward_kernel[grid_size, block_size](patches, self.kernels, convolution_output)
        cuda.synchronize()
        return convolution_output

    def back_prop(self, dE_dY, alpha):
        """
        Takes the gradient of the loss function with respect to the output and computes the gradients of the loss function with respect
        to the kernels' weights.
        dE_dY comes from the following layer, typically max pooling layer.
        It updates the kernels' weights
        """
        # Initialize gradient of the loss function with respect to the kernel weights
        dE_dk = np.zeros(self.kernels.shape)
        patches = self.patches_generator(self.image)
        block_size = (16, 4, 4)
        grid_size = (math.ceil(dE_dk.shape[2] / block_size[0]),
                     math.ceil(dE_dk.shape[1] / block_size[1]),
                     math.ceil(dE_dk.shape[0] / block_size[2]))
        cnn_backward_kernel[grid_size, block_size](patches, dE_dY, dE_dk)
        cuda.synchronize()
        # Update the parameters
        self.kernels -= alpha*dE_dk
        return dE_dk


class MaxPoolingLayer:
    def __init__(self, kernel_size):
        """
        Constructor takes as input the size of the kernel
        """
        self.kernel_size = kernel_size
    @cuda.jit
    def patches_generator(self, image):
        """
        Divide the input image in patches to be used during pooling.
        Yields the tuples containing the patches and their coordinates.
        """
        # Compute the ouput size
        output_h = image.shape[0] // self.kernel_size
        output_w = image.shape[1] // self.kernel_size
        self.image = image
        c,r=cuda.grid(2)
        if r < output_h and c < output_w:
              patch = image[(r*self.kernel_size):(r*self.kernel_size+self.kernel_size), (c*self.kernel_size):(c*self.kernel_size+self.kernel_size)]
              yield patch, r, c

    @cuda.jit
    def forward_prop(self, image):
        image_h, image_w, num_kernels = image.shape
        max_pooling_output = np.zeros((image_h//self.kernel_size, image_w//self.kernel_size, num_kernels))
        c,r=cuda.grid(2)
        block_size = (32, 32)
        output_h = image.shape[0] // self.kernel_size
        output_w = image.shape[1]
        grid_size = (math.ceil(image.shape[1] / block_size[0]),
             math.ceil(image.shape[0] / block_size[1]))
        if r < output_h and c < output_w:
            for patch, r, c in self.patches_generator[grid_size, block_size](image):
                max_pooling_output[r,c] = np.amax(patch, axis=(0,1))
            return max_pooling_output


    def back_prop(self, dE_dY):
        """
        Takes the gradient of the loss function with respect to the output and computes the gradients of the loss function with respect
        to the kernels' weights.
        dE_dY comes from the following layer, typically softmax.
        There are no weights to update, but the output is needed to update the weights of the convolutional layer.
        """
        dE_dk = np.zeros(self.image.shape)
        for patch,h,w in self.patches_generator(self.image):
            image_h, image_w, num_kernels = patch.shape
            max_val = np.amax(patch, axis=(0,1))

            for idx_h in range(image_h):
                for idx_w in range(image_w):
                    for idx_k in range(num_kernels):
                        if patch[idx_h,idx_w,idx_k] == max_val[idx_k]:
                            dE_dk[h*self.kernel_size+idx_h, w*self.kernel_size+idx_w, idx_k] = dE_dY[h,w,idx_k]
            return dE_dk

class SoftmaxLayer:
    """
    Takes the volume coming from convolutional & pooling layers. It flattens it and it uses it in the next layers.
    """
    def __init__(self, input_units, output_units):
        # Initiallize weights and biases
        self.weight = np.random.randn(input_units, output_units)/input_units
        self.bias = np.zeros(output_units)

    def forward_prop(self, image):
      self.original_shape = image.shape # stored for backprop
      # Flatten the image
      #print("image: ", image)
      image_flattened = image.flatten()
      #print("image_flattened: ", image_flattened)
      self.flattened_input = image_flattened # stored for backprop

      # Perform matrix multiplication and add bias
      C = np.empty(10)
      dA = cuda.to_device(image_flattened)
      dB = cuda.to_device(self.weight)
      dC = cuda.to_device(C)
      dot[(self.weight.shape[0]+255)//256, 256](dA,dB,dC)
      #cu_matrix_vector[(dZ_dX.shape[0]+511)//512, 512](dZ_dX,dE_dZ,C)
      result = dC.copy_to_host()
      first_output = result  + self.bias
      self.output = first_output
      # Apply softmax activation
      softmax_output = np.exp(first_output) / np.sum(np.exp(first_output), axis=0)

      return softmax_output


    def back_prop(self, dE_dY, alpha):
      for i, gradient in enumerate(dE_dY):
        if gradient == 0:
          continue
        transformation_eq = np.exp(self.output)
        S_total = np.sum(transformation_eq)

        # Compute gradients with respect to output (Z)
        dY_dZ = -transformation_eq[i]*transformation_eq / (S_total**2)
        dY_dZ[i] = transformation_eq[i]*(S_total - transformation_eq[i]) / (S_total**2)

        # Compute gradients of output Z with respect to weight, bias, input
        dZ_dw = self.flattened_input
        dZ_db = 1
        dZ_dX = self.weight

        # Gradient of loss with respect ot output
        dE_dZ = gradient * dY_dZ

        # Gradient of loss with respect to weight, bias, input
        dE_dw = dZ_dw[np.newaxis].T @ dE_dZ[np.newaxis]
        dE_db = dE_dZ * dZ_db

        # Matrix-vector multiply function
        C = np.empty(dZ_dX.shape[0])
        dA = cuda.to_device(dZ_dX)
        dB = cuda.to_device(dE_dZ)
        dC = cuda.to_device(C)
        cu_matrix_vector[(dZ_dX.shape[0]+127)//128, 128](dA,dB,dC)
        dE_dX = dC.copy_to_host()

        # Update parameters
        self.weight -= alpha* (dZ_dw[np.newaxis].T @ dE_dZ[np.newaxis])
        self.bias -= alpha * (dE_dZ * dZ_db)

        return dE_dX.reshape(self.original_shape)


def CNN_forward(image, label, layers):
    output = image/255.
    for layer in layers:
      if type(layer) == MaxPoolingLayer:
        output = forward_prop(output,kernel_size=2)
      elif type(layer) == ConvolutionLayer:
        output = layer.forward_prop(output)
        image_backprob_max=output
      else:
        output = layer.forward_prop(output)
    # Compute loss (cross-entropy) and accuracy
    loss = -np.log(output[label])
    accuracy = 1 if np.argmax(output) == label else 0
    return output, loss, accuracy,image_backprob_max


def CNN_backprop(gradient, layers,image_backprob_max, alpha=0.05):
    grad_back = gradient
    for layer in layers[::-1]:
        if type(layer) in [ConvolutionLayer, SoftmaxLayer]:
            grad_back = layer.back_prop(grad_back, alpha)
        elif type(layer) == MaxPoolingLayer:
            grad_back = back_prop(image_backprob_max,grad_back,kernel_size=2)
    return grad_back


def CNN_training(image, label, layers, alpha=0.05):
    # Forward step
    output, loss, accuracy,image_backprob_max = CNN_forward(image, label, layers)

    # Initial gradient
    gradient = np.zeros(10)
    gradient[label] = -1/output[label]

    # Backprop step
    gradient_back = CNN_backprop(gradient, layers,image_backprob_max, alpha)

    return loss, accuracy




"""
import pandas as pd
import matplotlib.pyplot as plt
#Test the convolutions with 1 image, to put in the article
# Test
df_train = pd.read_csv('train.csv')
img = df_train.iloc[40,:].values[1:]
img = np.reshape(img,(28,28))
plt.imshow(img, cmap='gray')
plt.show()
print(img.shape)
plt.savefig('images/original_image.png', format='png', dpi=1200)

# Test with a convolution of 16 filters of size 3x3
my_conv = ConvolutionLayer(32,3)
output = my_conv.forward_prop(img)
# See the dimensions of the output volume, they follow the usual formula
print(output.shape)

# Plot 16th volume after the convolution
plt.imshow(output[:,:,15], cmap='gray')
plt.show()
plt.savefig('images/image_convolved.png', format='png', dpi=1200)
"""

"\nimport pandas as pd\nimport matplotlib.pyplot as plt\n#Test the convolutions with 1 image, to put in the article\n# Test\ndf_train = pd.read_csv('train.csv')\nimg = df_train.iloc[40,:].values[1:]\nimg = np.reshape(img,(28,28))\nplt.imshow(img, cmap='gray')\nplt.show()\nprint(img.shape)\nplt.savefig('images/original_image.png', format='png', dpi=1200)\n\n# Test with a convolution of 16 filters of size 3x3\nmy_conv = ConvolutionLayer(32,3)\noutput = my_conv.forward_prop(img)\n# See the dimensions of the output volume, they follow the usual formula\nprint(output.shape)\n\n# Plot 16th volume after the convolution\nplt.imshow(output[:,:,15], cmap='gray')\nplt.show()\nplt.savefig('images/image_convolved.png', format='png', dpi=1200)\n"

In [2]:
@cuda.jit
def cnn_forward_kernel(patches, kernels, convolution_output):
    r, c = cuda.grid(2)

    if r < patches.shape[0] and c < patches.shape[1]:
        for k in range(kernels.shape[0]):
            sum = 0
            for i in range(kernels.shape[1]):
                for j in range(kernels.shape[2]):
                    sum += patches[r, c, i, j] * kernels[k, i, j]
            convolution_output[r, c, k] = sum

@cuda.jit
def cnn_backward_kernel(patches, dE_dY, dE_dk):
    x, y, z = cuda.grid(3)

    if x < dE_dk.shape[0] and y < dE_dk.shape[1] and z < dE_dk.shape[2]:
        temp = 0
        for h in range(patches.shape[0]):
            for w in range(patches.shape[1]):
                temp += patches[h, w, y, z] * dE_dY[h, w, x]
        dE_dk[x, y, z] = temp

In [9]:
#@cuda.jit(device=True)
def patches_generator(image,kernel_size):
        """
        Divide the input image in patches to be used during pooling.
        Yields the tuples containing the patches and their coordinates.
        """
        # Compute the ouput size
        output_h = image.shape[0] // kernel_size
        output_w = image.shape[1] // kernel_size
        #self.image = image
        #c,r=cuda.grid(2)
        for h in range(output_h):
            for w in range(output_w):
                patch = image[(h*kernel_size):(h*kernel_size+kernel_size), (w*kernel_size):(w*kernel_size+kernel_size)]
                yield patch,h,w

def forward_prop(image,kernel_size):
        image_h, image_w, num_kernels = image.shape
        max_pooling_output = np.zeros((image_h//kernel_size, image_w//kernel_size, num_kernels))
        for patch, h, w in patches_generator(image,kernel_size):
            max_pooling_output[h,w] = np.amax(patch, axis=(0,1))
        return max_pooling_output



def back_prop(image,dE_dY,kernel_size):
        """
        Takes the gradient of the loss function with respect to the output and computes the gradients of the loss function with respect
        to the kernels' weights.
        dE_dY comes from the following layer, typically softmax.
        There are no weights to update, but the output is needed to update the weights of the convolutional layer.
        """
        dE_dk_temp1 = np.zeros(image.shape)
        #dE_dk=np.ascontiguousarray(dE_dk_temp1)
        #cuda.pinned(dE_dk)
        for patch,h,w in patches_generator(image,kernel_size):
            image_h, image_w, num_kernels = patch.shape
            max_val = np.amax(patch, axis=(0,1))
            block_size = (16, 16)
            grid_size = (math.ceil(image_w / block_size[0]),
                        math.ceil(image_h / block_size[1]))
            patch=np.ascontiguousarray(patch)

            dA=cuda.to_device(patch)
            #print("hihi2")
           # dB=cuda.to_device(max_val)
            #dC=cuda.to_device(dE_dY)
            dE=cuda.to_device(dE_dk_temp1)
            #print("hihi")
            back_prob_sup[grid_size, block_size](image_h,image_w,num_kernels,dA,max_val,dE_dY,dE,h,w,kernel_size)
            dE_dk=dE.copy_to_host()
            #print(dE_dk)
            #stream.synchronize()
            return dE_dk

@cuda.jit
def back_prob_sup(image_h,image_w,num_kernels,patch,max_val,dE_dY,dE_dk,h,w,kernel_size):
    c,r=cuda.grid(2)
    #print("hihi")
    if r < image_h and c < image_w:
          for idx_k in range(num_kernels):
                if patch[r,c,idx_k] == max_val[idx_k]:
                  #print("hihi")
                  dE_dk[h*kernel_size+r, w*kernel_size+c, idx_k] = dE_dY[h,w,idx_k]
    #return dE_dk

In [4]:
@cuda.jit
def dot(a, b, c):
  col = cuda.grid(1)
  if (col < b.shape[1]):
    sum = 0.0
    for i in range(b.shape[0]):
      sum += a[i] * b[i, col]
    c[col] = sum

@cuda.jit
def cu_matrix_vector(A, b, c):
  row = cuda.grid(1)
  if (row < A.shape[0]):
    sum = 0.0
    for i in range(A.shape[1]):
      sum += A[row, i] * b[i]
    c[row] = sum

In [10]:
import numpy as np
import math
from numba import cuda
#from utils import *
import tensorflow as tf
import time

def main():
  # Load training data
  (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
  X_train = X_train[:10000]
  y_train = y_train[:10000]

  # Define the network
  layers = [
    ConvolutionLayer(16,3), # layer with 8 3x3 filters, output (26,26,16)
    MaxPoolingLayer(2), # pooling layer 2x2, output (13,13,16)
    SoftmaxLayer(13*13*16, 10) # softmax layer with 13*13*16 input and 10 output
    ]

  for epoch in range(1):
    print('Epoch {} ->'.format(epoch+1))
    # Shuffle training data
    permutation = np.random.permutation(len(X_train))
    X_train = X_train[permutation]
    y_train = y_train[permutation]
    # Training the CNN
    loss = 0
    accuracy = 0
    for i, (image, label) in enumerate(zip(X_train, y_train)):
      if i % 100 == 0: # Every 100 examples
        print("Step {}. For the last 100 steps: average loss {}, accuracy {}".format(i+1, loss/100, accuracy))
        loss = 0
        accuracy = 0
      image_backprob_max=image
      loss_1, accuracy_1 = CNN_training(image, label, layers)
      loss += loss_1
      accuracy += accuracy_1


if __name__ == '__main__':
  start = time.time()
  main()
  end = time.time()
  print(f'Processing time: {end - start} s')

Epoch 1 ->
Step 1. For the last 100 steps: average loss 0.0, accuracy 0


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 11 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 22 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due 

Step 101. For the last 100 steps: average loss 1.9125924102695244, accuracy 38
Step 201. For the last 100 steps: average loss 1.1524619328957568, accuracy 61
Step 301. For the last 100 steps: average loss 0.8427925436771072, accuracy 75
Step 401. For the last 100 steps: average loss 0.6812027035574276, accuracy 81
Step 501. For the last 100 steps: average loss 0.609201943954803, accuracy 85
Step 601. For the last 100 steps: average loss 0.5241411681466849, accuracy 85
Step 701. For the last 100 steps: average loss 0.6334343813585073, accuracy 82
Step 801. For the last 100 steps: average loss 0.5803650026036764, accuracy 82
Step 901. For the last 100 steps: average loss 0.5479547773331299, accuracy 82
Step 1001. For the last 100 steps: average loss 0.3912323309632928, accuracy 91
Step 1101. For the last 100 steps: average loss 0.5351177270499027, accuracy 89
Step 1201. For the last 100 steps: average loss 0.32963504388009196, accuracy 87
Step 1301. For the last 100 steps: average loss 0